In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/apple-stock-data/apple_stock_data.csv


In [4]:
apple_stock_data = pd.read_csv('/kaggle/input/apple-stock-data/apple_stock_data.csv')

In [5]:
apple_stock_data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [6]:
apple_stock_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Adj Close  252 non-null    float64
 2   Close      252 non-null    float64
 3   High       252 non-null    float64
 4   Low        252 non-null    float64
 5   Open       252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


In [7]:
#Converting Date to datetime
apple_stock_data['Date'] = pd.to_datetime(apple_stock_data['Date'])

In [8]:
apple_stock_data.set_index('Date', inplace=True)

In [9]:
apple_stock_data = apple_stock_data[['Close']]

In [10]:
apple_stock_data

,Close
Date,
2023-11-02 00:00:00+00:00,177.570007
2023-11-03 00:00:00+00:00,176.649994
2023-11-06 00:00:00+00:00,179.229996
2023-11-07 00:00:00+00:00,181.820007
2023-11-08 00:00:00+00:00,182.889999
...,...
2024-10-28 00:00:00+00:00,233.399994
2024-10-29 00:00:00+00:00,233.669998
2024-10-30 00:00:00+00:00,230.100006


In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
apple_stock_data['Close'] = scaler.fit_transform(apple_stock_data[['Close']])

In [12]:
def create_sequence(data, seq_length= 60):
    X,y=[],[]
    for i in range(len(apple_stock_data) - seq_length ):
        X.append(apple_stock_data[i:i+seq_length])
        y.append(apple_stock_data[i:i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X,y = create_sequence(apple_stock_data['Close'].values, seq_length)

In [13]:
train_size = int(len(X) *0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense

2025-04-17 19:55:32.881979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744919733.131336      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744919733.201961      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

2025-04-17 19:56:00.293057: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
lstm_model.compile(optimizer= 'adam', loss='mean_squared_error')
lstm_model.fit(X_train,y_train, epochs= 20, batch_size= 32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.1263
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0451
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0494
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0337
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0347
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0346
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0336
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0345
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0313
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0328
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0322
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0319
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0322
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0318
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0306
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

In [17]:
apple_stock_data['Lag_1'] = apple_stock_data['Close'].shift(1)
apple_stock_data['Lag_2'] = apple_stock_data['Close'].shift(2)
apple_stock_data['Lag_3'] = apple_stock_data['Close'].shift(3)

In [18]:
apple_stock_data.dropna(inplace=True)

In [19]:
apple_stock_data.isnull().sum()

Close    0
Lag_1    0
Lag_2    0
Lag_3    0
dtype: int64

In [20]:
X_lin = apple_stock_data[['Lag_1',"Lag_2",'Lag_3']]
y_lin = apple_stock_data['Close']

In [21]:
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [22]:
X_train_lin.isnull().sum()

Lag_1    0
Lag_2    0
Lag_3    0
dtype: int64

In [23]:
X_train_lin.shape

(153, 3)

In [24]:
X_test_lin.shape

(96, 3)

In [25]:
y_train_lin.shape

(153,)

In [26]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train_lin,y_train_lin)

LinearRegression()

In [27]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 394ms/step


In [28]:
lin_predictions = lm.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [29]:
print(lstm_predictions.shape)
print(lin_predictions.shape)

(39, 1)
(96, 1)


In [32]:
min_len = min(len(lstm_predictions), len(lin_predictions))
hybrid_predictions = (0.7 * lstm_predictions[-min_len:]) + (0.3 * lin_predictions[-min_len:])

In [35]:
#Predicting for the next 10 days using LSTM
lstm_future_pred = []
last_sequence = X[-1].reshape(1, seq_length,1)

for _ in range(10):
    lstm_pred= lstm_model.predict(last_sequence)[0,0]
    lstm_future_pred.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1,1,1)
    last_sequence = np.append(last_sequence[:1,1:,:],lstm_pred_reshaped, axis = 1 )

lstm_future_pred = scaler.inverse_transform(np.array(lstm_future_pred).reshape(-1,1)) 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [37]:
#Using linear regression to predict the next 10 days

recent_data = apple_stock_data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lm.predict(recent_data.reshape(1,-1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1,1))

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

In [40]:
hybrid_future_predictions = (0.7 * lstm_future_pred[-min_len:]) + (0.3 * lin_future_predictions[-min_len:])

In [43]:
#Creating a dataframe to see the final predictions

future_dates = pd.date_range(start=apple_stock_data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_pred.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
print(predictions_df)

                       Date  LSTM Predictions  Linear Regression Predictions  \
0 2024-11-02 00:00:00+00:00        218.813141                     230.355192   
1 2024-11-03 00:00:00+00:00        218.914307                     225.707291   
2 2024-11-04 00:00:00+00:00        218.989182                     222.703426   
3 2024-11-05 00:00:00+00:00        219.040787                     230.631535   
4 2024-11-06 00:00:00+00:00        219.062271                     225.486380   
5 2024-11-07 00:00:00+00:00        219.069519                     222.494588   
6 2024-11-08 00:00:00+00:00        219.055084                     230.930195   
7 2024-11-09 00:00:00+00:00        219.027603                     225.245599   
8 2024-11-10 00:00:00+00:00        218.994354                     222.284007   
9 2024-11-11 00:00:00+00:00        218.953262                     231.252375   

   Hybrid Model Predictions  
0                222.275747  
1                220.952193  
2                220.103447  

**So, this is how to build a hybrid machine learning model using Python. Build a hybrid machine learning model when a single algorithm cannot capture the complexity of the data or when different types of data or patterns are present**